# 手把手教你写 SelfAttV4 (Step-by-Step)

你好！本教程将带你把提供的 `SelfAttV4` 代码拆解开来，一步步实现。

**💡 互动方式**：
如果你在阅读代码或运行过程中有任何疑问，请直接在代码单元格中添加注释提问，例如：
```python
# ? 为什么这里要除以 sqrt(dim)？
# Question: 这个 dropout 是必须的吗？
```
我会读取你的注释并直接在 notebook 中回复你。

## 第一步：导入与准备

首先导入必要的库。`math` 用于计算平方根，`torch` 是核心框架。

In [ ]:
import math
import torch
import torch.nn as nn
import warnings

warnings.filterwarnings(action="ignore")

# 设置随机种子以便复现结果
torch.manual_seed(42)

print("环境准备就绪！")

## 第二步：定义模型结构 (__init__)

在初始化函数中，我们定义所有的层。你的代码中使用了三个独立的线性层分别映射 Q, K, V，这种写法非常清晰。

我们还定义了 `output_proj` 和 `att_drop`。

In [ ]:
class SelfAttV4_Init(nn.Module):
    def __init__(self, dim) -> None:
        super().__init__()
        self.dim = dim   

        # 1. 定义 Q, K, V 的投影层
        # 这样很清晰，每个都有独立的参数
        self.query_proj = nn.Linear(dim, dim)
        self.key_proj = nn.Linear(dim, dim)
        self.value_proj = nn.Linear(dim, dim)

        # 2. 定义 Dropout
        # 一般是 0.1 的 dropout
        self.att_drop = nn.Dropout(0.1)

        # 3. 定义输出投影
        # 这是 MultiHeadAttention 中的产物，即使是单头也可以保留
        self.output_proj = nn.Linear(dim, dim)

    def forward(self, X, attention_mask=None):
        # 暂时留空，下一步实现
        pass

# 测试初始化
model_init = SelfAttV4_Init(dim=2)
print(model_init)

## 第三步：计算 Attention Scores

这是最核心的步骤：
1. 通过投影层得到 Q, K, V。
2. 计算 $QK^T$。
3. 除以 $\sqrt{d}$ 进行缩放 (Scale)。

In [ ]:
def calculate_scores(self, X):
    # X shape: (batch, seq, dim)
    
    # 1. 投影
    Q = self.query_proj(X)
    K = self.key_proj(X)
    V = self.value_proj(X)
    
    # 2. 矩阵乘法 Q @ K^T
    # K.transpose(-1, -2) 将最后两个维度交换，(batch, seq, dim) -> (batch, dim, seq)
    # 结果 shape: (batch, seq, seq)
    att_weight = Q @ K.transpose(-1, -2) 
    
    # 3. 缩放
    att_weight = att_weight / math.sqrt(self.dim)
    
    return att_weight, V

# 这里的代码仅作演示逻辑，不能直接运行，因为它依赖 self

## 第四步：Masking (掩码)

如果有 `attention_mask`，我们需要把 mask 为 0 的位置替换成一个极小的数（例如 `-1e20`），这样在 Softmax 后它们会变成 0，表示不关注这些位置。

In [ ]:
def apply_mask(att_weight, attention_mask):
    if attention_mask is not None:
        # 给 weight 填充一个极小的值
        # attention_mask == 0 的位置会被填充
        att_weight = att_weight.masked_fill(attention_mask == 0, float("-1e20"))
    return att_weight

## 第五步：Softmax, Dropout 与 Output

1. **Softmax**: 将分数归一化为概率。
2. **Dropout**: 这是一个比较特殊的操作，在 BERT 官方代码中出现过，虽然有些奇怪，但保留它是为了对齐。
3. **Weighted Sum**: `Attention * V`。
4. **Output Project**: 最后的线性层。

In [ ]:
def finalize(self, att_weight, V):
    # 1. Softmax
    att_weight = torch.softmax(att_weight, dim=-1)
    
    # 打印一下看看权重分布
    # print(att_weight) 

    # 2. Dropout (特殊位置)
    # 这里在 BERT中的官方代码也说很奇怪，但是原文中这么用了，所以继承了下来
    att_weight = self.att_drop(att_weight)

    # 3. 加权求和
    output = att_weight @ V
    
    # 4. 输出投影
    ret = self.output_proj(output)
    return ret

## 完整代码：SelfAttV4

现在我们将所有步骤整合到一起，这正是你提供的完整代码。

In [ ]:
class SelfAttV4(nn.Module):
    def __init__(self, dim) -> None:
        super().__init__()
        self.dim = dim

        # 这样很清晰
        self.query_proj = nn.Linear(dim, dim)
        self.key_proj = nn.Linear(dim, dim)
        self.value_proj = nn.Linear(dim, dim)
        # 一般是 0.1 的 dropout，一般写作 config.attention_probs_dropout_prob
        # hidden_dropout_prob 一般也是 0.1
        self.att_drop = nn.Dropout(0.1)

        # 可以不写；具体和面试官沟通。
        # 这是 MultiHeadAttention 中的产物，这个留给 MultiHeadAttention 也没有问题；
        self.output_proj = nn.Linear(dim, dim)

    def forward(self, X, attention_mask=None):
        # attention_mask shape is: (batch, seq)
        # X shape is: (batch, seq, dim)

        Q = self.query_proj(X)
        K = self.key_proj(X)
        V = self.value_proj(X)

        att_weight = Q @ K.transpose(-1, -2) / math.sqrt(self.dim)
        if attention_mask is not None:
            # 给 weight 填充一个极小的值
            att_weight = att_weight.masked_fill(attention_mask == 0, float("-1e20"))

        att_weight = torch.softmax(att_weight, dim=-1)
        print("Attention Weights:", att_weight)

        # 这里在 BERT中的官方代码也说很奇怪，但是原文中这么用了，所以继承了下来
        # （用于 output 后面会更符合直觉？）
        att_weight = self.att_drop(att_weight)

        output = att_weight @ V
        ret = self.output_proj(output)
        return ret

## 运行测试

最后，我们使用你提供的测试数据来运行模型。

In [ ]:
X = torch.rand(3, 4, 2)
b = torch.tensor(
    [
        [1, 1, 1, 0],
        [1, 1, 0, 0],
        [1, 0, 0, 0],
    ]
 )
print("b shape:", b.shape)

# 构造 mask
# b.unsqueeze(dim=1) -> (3, 1, 4)
# .repeat(1, 4, 1) -> (3, 4, 4)
mask = b.unsqueeze(dim=1).repeat(1, 4, 1)
print("Mask shape:", mask.shape)

net = SelfAttV4(2)
output = net(X, mask)
print("Output shape:", output.shape)